# ⭐ WhisperJAV transciber by a-pikachu

---



This is based on the work of [meizhong986](https://github.com/meizhong986/WhisperJAV). Thank you!


In order to accelerate transcribe, you can strip the video track off the mp4 with ffmpeg

1.   ffmpeg -i <video.mp4> -vn -acodec copy <audio-only.mp4>
2.   Upload the audio only mp4 to Google Drive/WhisperJAV/input


In [ ]:
#@title Install WhisperJAV
!nvidia-smi

!sudo apt-get install python3.10
!sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.10 1

# Install Whisper + WhisperJAV
!pip install --upgrade pip

# Install required dependencies manually
!pip install pyloudnorm soundfile pydub librosa numpy
!pip install openai-whisper ffmpeg-python pysrt srt stable-ts faster-whisper[cuda]
!pip install auditok --no-deps

# Install WhisperJAV WITHOUT dependencies (avoids PyAudio)
!pip install --no-deps --force-reinstall git+https://github.com/meizhong986/WhisperJAV.git

# Ensure ffmpeg is available
!apt-get update
!apt-get install -y ffmpeg

In [ ]:
#@title Checking WhisperJAV installation
import whisperjav
print("WhisperJAV version:", whisperjav.__version__)

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#@title Listing available video under /content/drive/MyDrive/WhisperJAV/input
import os
import ipywidgets as widgets
from IPython.display import display

# Path to your input folder
input_folder = "/content/drive/MyDrive/WhisperJAV/input"

# List all mp4 files
video_files = [f for f in os.listdir(input_folder) if f.lower().endswith(".mp4")]

# Create dropdown widget
video_dropdown = widgets.Dropdown(
    options=video_files,
    description='Select Video:',
    layout=widgets.Layout(width='600px')
)

display(video_dropdown)

In [ ]:
#@title Confirm Video Filename
video_file = os.path.join(input_folder, video_dropdown.value)
print("Selected video:", video_file)

In [ ]:
%%writefile whisperjav_patched.py
import subprocess
import whisperjav.modules.audio_extraction as ae

# Save original run
_original_run = subprocess.run

def safe_run(cmd, *args, **kwargs):
    # Only intercept FFmpeg calls inside audio_extraction
    if isinstance(cmd, list) and cmd and 'ffmpeg' in cmd[0]:
        return _original_run(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=False,      # prevent UTF‑8 decoding
        )
    # Everything else uses the original behavior
    return _original_run(cmd, *args, **kwargs)

# Patch ONLY inside audio_extraction
ae.subprocess.run = safe_run

# Now import WhisperJAV CLI entry point
from whisperjav.main import main

if __name__ == "__main__":
    main()

In [ ]:
#@title Transcribe Video
import os

output_dir = "/content/drive/MyDrive/WhisperJAV/output"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print("Created output folder:", output_dir)
else:
    print("Output folder already exists:", output_dir)


!python3 whisperjav_patched.py "$video_file" --output-dir /content/drive/MyDrive/WhisperJAV/output


